In [1]:
import torch
import random
import torchvision
import numpy as np
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import transforms
from sklearn.model_selection import train_test_split

from utils.util import get_image_list, dataset_list, PathDataset, PathDataset_v2, make_task

from avalanche.training.supervised import LwF
from avalanche.logging import InteractiveLogger, WandBLogger
from avalanche.benchmarks.generators import nc_benchmark
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import ExperienceAccuracy, ExperienceLoss, ExperienceForgetting, ExperienceCPUUsage, ExperienceMaxGPU, ExperienceMaxRAM, ExperienceTime, EpochAccuracy

from avalanche.evaluation.metrics import (
    forgetting_metrics,
    accuracy_metrics,
    loss_metrics
)

In [2]:
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
# torch.cuda.manual_seed_all(seed) # if use multi-GPU
cudnn.deterministic = True  # 연산 처리 속도 감소 -> 모델과 코드를 배포해야 하는 연구 후반 단계에 사용
cudnn.benchmark = False

In [3]:
data_path = "../Data/CASIA-WebFace/" 

dataset = get_image_list(data_path)
x_train, x_test, y_train, y_test = train_test_split(dataset[0], dataset[1], test_size=0.2, shuffle=True, random_state=seed, stratify=dataset[1])

train_set = dataset_list(x_train, y_train)
test_set = dataset_list(x_test, y_test)

print("train : ", len(y_train))
print("test : ", len(y_test))

train :  395531
test :  98883


In [4]:
transforms_group = dict(
       train=(
       transforms.Compose(
              [
              transforms.ToTensor(),
              ]
       ),
       None,
       ),
       eval=(
       transforms.Compose(
              [
              transforms.ToTensor(),
              ]
       ),
       None,
       )
)

In [5]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

num_class = 10575
incremental = 5
train_task = make_task(incremental, y_train)
test_task = make_task(incremental, y_test)

train_set = PathDataset(train_set, transforms_group, "train", train_task)
test_set = PathDataset(test_set, transforms_group, "eval", test_task)
# train_set = PathDataset_v2(train_set, transforms_group, "train")
# test_set = PathDataset_v2(test_set, transforms_group, "eval")

scenario = nc_benchmark(train_dataset=train_set,
                        test_dataset=test_set,
                        n_experiences=incremental,
                        task_labels=True,
                        seed=seed,
                        shuffle=False
                        )

model = torchvision.models.resnet18(pretrained=False, num_classes=num_class)
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=1e-3)
criterion = torch.nn.CrossEntropyLoss()

In [6]:
interactive_logger = InteractiveLogger()
wandb_logger = WandBLogger(project_name="Face Recognition", run_name="LwF-CASIA-v2")
eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    forgetting_metrics(experience=True),
    ExperienceTime(),
    loggers=[interactive_logger, wandb_logger])

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hopo55. Use `wandb login --relogin` to force relogin


/home/cal-06/anaconda3/envs/hspark/lib/python3.8/site-packages/avalanche/training/plugins/evaluation.py:81: UserWarning: No benchmark provided to the evaluation plugin. Metrics may be computed on inconsistent portion of streams, use at your own risk.
  warnings.warn(


## LwF

In [7]:
train_batch = 10
eval_batch = 1
epoch = 10
lwf_alpha = [0, 0.5, 1.333, 2.25, 3.2]

strategies = LwF(model, optimizer, criterion, alpha=lwf_alpha, temperature=1, train_epochs=epoch, device=device, train_mb_size=train_batch, eval_mb_size=eval_batch, evaluator=eval_plugin)

In [8]:
print("Starting experiment...")
results = []
for experience in scenario.train_stream:
    print("Start training on experience ", experience.current_experience)
    strategies.train(experience)
    print("End training on experience ", experience.current_experience)
    print("Computing accuracy on the test set")
    results.append(strategies.eval(scenario.test_stream[:]))

Starting experiment...
Start training on experience  0
-- >> Start of training phase << --
100%|██████████| 914/914 [07:54<00:00,  1.93it/s]
Epoch 0 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0012
100%|██████████| 914/914 [08:06<00:00,  1.88it/s]
Epoch 1 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0056
100%|██████████| 914/914 [08:15<00:00,  1.84it/s]
Epoch 2 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0055
100%|██████████| 914/914 [08:13<00:00,  1.85it/s]
Epoch 3 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0057
100%|██████████| 914/914 [08:10<00:00,  1.86it/s]
Epoch 4 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0070
100%|██████████| 914/914 [08:16<00:00,  1.84it/s]
Epoch 5 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0089
100%|██████████| 914/914 [08:17<00:00,  1.84it/s]
Epoch 6 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0102
100%|██████████| 914/914 [08:22<00:00,  1.82it/s

KeyboardInterrupt: 